In [1]:
import numpy as np
import torch
import librosa
import librosa.display
import pandas as pd
from IPython.display import Audio
import seaborn as sns
import os

from collections import defaultdict

import tqdm
sns.set()

import matplotlib.pyplot as plt
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
import utils

%load_ext autoreload
%autoreload 2

In [2]:
import utils
import models
import trainer

In [72]:
class conf:
    # Preprocessing settings
    sampling_rate = 44100
    duration = 2
    hop_length = 347*duration # to make time steps 128
#     hop_length = 694*duration
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration
    trim_long_data = True

In [73]:
def show(audio):
    return Audio(audio, rate=conf.sampling_rate)

In [74]:
conf.columns, conf.column_encoder = utils.read_columns()

In [75]:
all_data, all_y, max_len = utils.read_dataset("./data/train_curated", conf, lambda s: s)


 83%|████████▎ | 4133/4970 [00:35<00:08, 104.47it/s]


100%|██████████| 4970/4970 [00:42<00:00, 98.34it/s] 

In [77]:
lengths = np.array(list([len(a) for a in all_data]))/conf.sampling_rate

In [78]:
np.quantile(lengths, 0.95)

2.0

In [32]:
all_data = np.array(all_data)

In [115]:
show(all_data[234] + 10 * all_data[1353] + all_data[1334] + 0.1 * np.random.randn(*all_data[0].shape))

In [66]:
all_data[lengths > 45][0].shape[0]/conf.sampling_rate

56.79600907029479

In [67]:
conf.columns[np.nonzero(all_y[lengths > 45][0])]

Index(['Stream'], dtype='object')

In [110]:
all_data[0].max()

0.0932312

In [122]:
x, y = trainer.mix_sounds(all_data, all_y, 5)

In [123]:
show(x)

In [148]:
loader = trainer.SoundAugDataset(all_data, all_y, lambda s: utils.audio_to_melspectrogram(conf, s), 10000, 5, 42)